Baby's First Reinforcement Learning
----------------------------------------------------

In this notebook we will begin to explore the fundamentals of reinforcement learning. From Wikipedia:

**Reinforcement learning (RL)** is an area of machine learning concerned with how software agents ought to take actions in an environment so as to maximize some notion of cumulative reward. (see: https://en.wikipedia.org/wiki/Reinforcement_learning)

Reinforcement learning has seen numerous successes in recent years. Groups such as OpenAI and
Google Deepmind have successfully trained agents to play Atari games 1 with super-human skill. More recently,
groups have trained agents to play considerably more complex games like Dota II. In each of these cases, the AI
agent is trained by allowing it to explore the environment, and discovering on it’s own which sequences of
actions lead to maximization of the chosen reward.

We will begin by exploring an old idea known as a **Markov Decision Process** (MDP). MDP's can be applied in reinforcement learning contexts, but as we will see they have limitations. If time allows, we will also explore **Deep Reinforcement Learning** which is a collection of newer techniques designed to overcome the limitations of MDP's by using **Neural Networks**.

In [1]:
#Step 1: Set up the environment such that we can view it, modify it, determine rewards

#Step 2: Initialize trainable MDP algorithm

#Step 3: Run trainable MDP algorithm on the environment from step 1. Observe reward/time curve.